In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import re

In [2]:
pattern = re.compile(r'(\d+):(\d+)\.(\d+)')
def to_time(s):
    m = pattern.search(s)
    if int(m.group(3)) > 10:
        return int(m.group(1))*60 + int(m.group(2)) + int(m.group(3))*0.001
    else:
        return int(m.group(1))*60 + int(m.group(2)) + int(m.group(3))*0.1

In [3]:
def DealWithTime(data):
#     data['time'] = data['timestamp'].apply(to_time)
#     return data.drop('timestamp',axis=1)
    data['time'] = pd.to_datetime(data['timestamp'])
    return data

In [4]:
def GetData(filename):
    data = pd.read_csv(filename)
    data = data[['loggingTime(txt)','motionUserAccelerationX(G)', 'motionUserAccelerationY(G)', 'motionUserAccelerationZ(G)',\
                 'gyroRotationX(rad/s)', 'gyroRotationY(rad/s)', 'gyroRotationZ(rad/s)',\
                 'motionYaw(rad)', 'motionRoll(rad)', 'motionPitch(rad)']]
    colums_name = ['timestamp','accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', 'yaw','roll', 'pitch']
    data.columns = colums_name
    return data

In [5]:
def Overview(data):
    plt.figure(figsize=(15,20))
    p1 = plt.subplot(3,1,1)
    p2 = plt.subplot(3,1,2)
    p3 = plt.subplot(3,1,3)
    p1.plot(left['accX'])
    p2.plot(left['accY'])
    p3.plot(left['accZ'])
    plt.show()

    plt.figure(figsize=(15,20))
    p4 = plt.subplot(3,1,1)
    p5 = plt.subplot(3,1,2)
    p6 = plt.subplot(3,1,3)
    p4.plot(left['gyroX'])
    p5.plot(left['gyroY'])
    p6.plot(left['gyroZ'])
    plt.show()

In [6]:
def butter_bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def butter_lowpass(highcut, fs, order=3):
    nyq = 0.5 * fs
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='lowpass')
    return b, a

def butter_lowpass_filter(data, highcut, fs, order=3):
    b, a = butter_lowpass(highcut, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def getStepFrequency(data):
    ffted = np.fft.fft(data)
    freqs = np.fft.fftfreq(len(ffted))
    idx = np.argmax(np.abs(ffted))
    freq = freqs[idx]
    return abs(freq*fs)

fs = 30

In [7]:
left = GetData("../dataset/zmq_left_2017-11-27_14-31-39_-0800.csv")
right =  GetData("../dataset/zmq_right_2017-11-27_14-31-48_-0800.csv")
# left = GetData("../dataset/lzy_left_2017-11-27_14-36-05_-0800.csv")
# right =  GetData("../dataset/lzy_right_2017-11-27_14-36-02_-0800.csv")

In [8]:
left = DealWithTime(left)
right = DealWithTime(right)

In [10]:
right = right[500:1000]
right = right.reset_index(drop=True)
# Overview(right)

In [ ]:
left = left[500:1000]
left = left.reset_index(drop=True)
#Overview(left)

In [ ]:
# highcut = 5.5
# filtered = butter_lowpass_filter(left['accY'],highcut, fs)
# plt.figure(figsize=(10,5))
# plt.plot(filtered)
# plt.show()
# plt.figure(figsize=(10,5))
# plt.plot(np.fft.rfft(filtered))
# plt.show()
# plt.figure(figsize=(10,5))
# plt.plot(left['accY'])
# plt.show()
# plt.figure(figsize=(10,5))
# plt.plot(np.fft.rfft(left['accY']))
# plt.show()

In [ ]:
# lowcut = 0.08
# highcut = 6
# filtered = butter_bandpass_filter(left['gyroZ'],lowcut, highcut, fs)
# plt.figure(figsize=(10,5))
# plt.plot(filtered)
# plt.show()
# plt.figure(figsize=(10,5))
# plt.plot(np.fft.rfft(filtered))
# plt.show()
# plt.figure(figsize=(10,5))
# plt.plot(left['gyroZ'])
# plt.show()

In [ ]:
highcut = 5.5
left['accX'] = butter_lowpass_filter(left['accX'],highcut, fs)
left['accY'] = butter_lowpass_filter(left['accY'],highcut, fs)
left['accZ'] = butter_lowpass_filter(left['accZ'],highcut, fs)

In [ ]:
lowcut = 0.08
highcut = 6
left['gyroX'] = butter_bandpass_filter(left['gyroX'],lowcut, highcut, fs)
left['gyroY'] = butter_bandpass_filter(left['gyroY'],lowcut, highcut, fs)
left['gyroZ'] = butter_bandpass_filter(left['gyroZ'],lowcut, highcut, fs)

In [ ]:
# print(getStepFrequency(left['accY']))
# print(getStepFrequency(left['accX']))
# print(getStepFrequency(left['accZ']))
# print(getStepFrequency(left['gyroY']))
# print(getStepFrequency(left['gyroX']))
# print(getStepFrequency(left['gyroZ']))

In [ ]:
step_frequency = getStepFrequency(left['gyroX'])

In [ ]:
step_cycle = int(round(fs/step_frequency))

In [ ]:
# def seperateEachStep(data, step_cycle):
#     steps = pd.DataFrame()
#     for i in range(0, data.shape[0], step_cycle):
#         idx = np.argmin(data['gyroZ'].iloc[i:i+step_cycle])
#         steps = steps.append(data.iloc[idx])
#     return steps
# def seperateEachStep(data, step_cycle):
#     steps = pd.DataFrame()
#     idx_list = []
#     for i in range(0, data.shape[0], step_cycle):
#         idx = np.argmin(data['gyroZ'].iloc[i:i+step_cycle])
#         if(idx != data.shape[0] - 1):
#             idx_list.append(idx)
# #         steps = steps.append(data.iloc[idx])
# #     print(idx_list)
#     for i in range(0, len(idx_list)-1):
#         prev = idx_list[i]
#         nxt = idx_list[i+1]
#         interval = nxt - prev
#         floor1 = int(prev + interval /3)
#         floor2 = int(nxt - interval /3)
#         if(floor1 != floor2):
#             idx = np.argmin(np.abs(data['gyroZ'].iloc[floor1:floor2]))
#             steps = steps.append(data.iloc[idx])
#     return steps

def seperateEachStep(data, step_cycle):
    steps = pd.DataFrame()
    idx_list = []
    for i in range(0, data.shape[0], step_cycle):
        idx = np.argmin(data['gyroZ'].iloc[i:i+step_cycle])
        if(idx != data.shape[0] - 1):
            idx_list.append(idx)
    print(idx_list)
    for i in range(0, len(idx_list)-1):
        prev = idx_list[i]
        nxt = idx_list[i+1]
        while (data['gyroZ'].iloc[prev]<data['gyroZ'].iloc[prev+1]):
            prev +=1
        while (data['gyroZ'].iloc[nxt]<data['gyroZ'].iloc[nxt-1]):
            nxt -=1
        floor1 = prev;
        floor2 = nxt;
        if(floor1 < floor2):
            idx = np.argmin(data['gyroZ'].iloc[floor1:floor2])
            steps = steps.append(data.iloc[idx])
    return steps

In [ ]:
steps = seperateEachStep(left, step_cycle)
plt.figure(figsize=(10,5))
plt.plot(steps['gyroZ'],'r*',left['gyroZ'], 'b--')
plt.show()
plt.figure(figsize=(10,5))
plt.plot(steps['accY'],'r*',left['accY'], 'b--')
plt.show()

In [ ]:
steps['initVy'] = 0
steps['initVx'] = 0.10* steps['gyroZ']
steps['endVy'] = 0
steps['endVx'] = steps['initVx'].shift(-1)

In [ ]:
steps['begin_idx'] = steps.index
steps['end_idx'] = steps['begin_idx'].shift(-1)
steps = steps.dropna()
steps['end_idx'] = steps['end_idx'].astype(int)

In [ ]:
def calcSpeedY(data, begin, end):
    return data.loc[begin:end, 'accY'].sum()/fs
def calcSpeedZ(data, begin, end):
    return data.loc[begin:end, 'accZ'].sum()/fs
def calcSpeedX(data, begin, end):
    return data.loc[begin:end, 'accX'].sum()/fs

In [ ]:
steps['calcVy'] = steps.apply((lambda row: calcSpeedY(left, row['begin_idx'], row['end_idx'])), axis=1)
steps['calcVz'] = steps.apply((lambda row: calcSpeedZ(left, row['begin_idx'], row['end_idx'])), axis=1)
steps['calcVx'] = steps.apply((lambda row: calcSpeedX(left, row['begin_idx'], row['end_idx'])), axis=1)

In [ ]:
steps['deltaVy'] = steps['endVy'] - steps['calcVy']
steps['deltaVx'] = steps['endVx'] - steps['calcVx'] 

In [ ]:
steps['deltaT'] = (steps['end_idx'] - steps['begin_idx'])/fs

In [ ]:
steps = steps.reset_index(drop=True)

In [ ]:
begin = steps['begin_idx'].iloc[0]
end = steps['end_idx'].iloc[-1]
left_data = left[begin: end]
left_data['Vy'] = np.nan
left_data['Vx'] = np.nan

In [ ]:
def CorrectedV(begin, end, deltaVy, deltaVx, deltaT, initVy, initVx, data):
    data.loc[begin,'Vy'] = initVy
    data.loc[begin,'Vx'] = initVx
    for i in range(int(begin+1), int(end)):
        data.loc[i,'Vy'] = data.loc[i-1,'Vy']+ (data.loc[i-1,'accY'] + deltaVy/deltaT)/fs
        data.loc[i,'Vx'] = data.loc[i-1,'Vx']+ (data.loc[i-1,'accX'] + deltaVx/deltaT)/fs

In [ ]:
steps.apply((lambda row: CorrectedV(row['begin_idx'], row['end_idx'], row['deltaVy'], row['deltaVx'], row['deltaT'],\
                                   row['initVy'], row['initVx'], left_data)), axis=1)

In [ ]:
# def getFC(data, begin, end):
#     return data.loc[begin:end, 'Vy'].sum()/fs
def calcY(data, begin, end):
    return data.loc[begin:end, 'Vy'].sum()/fs

def CorrectedY(begin, end, calcY, deltaT, data):
    data.loc[begin,'correctY'] = 0
    for i in range(int(begin+1), int(end)):
        data.loc[i,'correctY'] = data.loc[i-1,'correctY']+ (data.loc[i-1,'Vy'] - calcY/deltaT)/fs
        
def getFC(data, begin, end):
    idx = np.argmin(data.loc[begin:end, 'correctY'])
    return data.loc[idx, 'correctY']

In [ ]:
steps['calcY'] = steps.apply((lambda row: calcY(left_data, row['begin_idx'], row['end_idx'])), axis=1)
# print(steps)

In [ ]:
steps.apply((lambda row: CorrectedY(row['begin_idx'], row['end_idx'], row['calcY'], row['deltaT'],left_data)), axis=1)
# print(left_data)

In [ ]:
steps['FC'] = steps.apply((lambda row: getFC(left_data, row['begin_idx'], row['end_idx'])), axis=1)
# print(steps)

In [ ]:
plt.plot(left_data['time'], left_data['Vy'], 'b-')
plt.plot(steps['time'], steps['endVy'], 'r*')
plt.show()
plt.plot(steps['FC'], 'b.')
plt.show()

In [ ]:
plt.plot(left_data['time'], left_data['Vx'], 'b-')
plt.plot(steps['time'], steps['initVx'], 'r*')
plt.show()